## Lobes

- Measure fluxes from different regions at multiple frequencies
- The r.m.s. in the input images should be measured in a different notebook and saved to the input table, since for the Planck images we use a different method.
- MeerKAT observations must be corrected for the primary beam BEFORE measuring the flux
- Planck observations must be converted to the correct units BEFORE measuring the flux
- **Delete** old flux tables before repeaing measurement

  - aut= 'Fg' -> original data
  - aut= 'FgCv' -> convolved datasets


In [1]:
import numpy as np
import shutil
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
from prettytable import PrettyTable



fint = fluxInt.flInt()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/'
#rootDir = '/Users/maccagni/Projects/MFS/FornaxA/contData/'


imDir = rootDir+'inIms/'
regDir = imDir+'regions/'
inTabs = rootDir+'inTabs/'

names = ['ToT','ELobe','WLobe']

ToTDir = rootDir+names[0]+'/'
ELobeDir = rootDir+names[1]+'/'
WLobeDir = rootDir+names[2]+'/'

aut = 'Fg'

synToTtable = ToTDir+'synageFl'+'ToT'+aut+'.tbl'
synETable = ELobeDir+'synageFl'+'ELobe'+aut+'.tbl'
synWTable = WLobeDir+'synageFl'+'WLobe'+aut+'.tbl'

fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir


tableName = inTabs+'inputFluxTable'+aut+'.tbl'
tableFileNames= ascii.read(tableName,format='csv')

noiseTableName = inTabs+'noiseFluxInt'+aut+'.tbl'
noiseValues= ascii.read(noiseTableName)

print tableFileNames.columns[0]

# always check that frequencies correspond to input files

#frList = [ 84.0,114.5,200.5, 320.0,1029.9794921,1440.5292968,1500.0,
#      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0,100000.0,143000.0]
frList = [ 84.0,114.5,155,200.5, 320.0,1029.9794921,1440.5292968,1500.0,
      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0,
          30000.0,44000.0,70000.0,100000.0,143000.0, 217000.0,353000.0,545000.0,857000.0]

#frList = [1029.9794921,1440.5292968,1500.0,
#      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0]
print frList
print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


IOError: [Errno 2] No such file or directory: '/home/maccagni/Projects/MFS/FornaxA/contData/inTabs/inputFluxTableFg.tbl'

We measure the flux in 24 frequency intervals  between 84 MHz and 217 GHz.

In [ ]:
totReg = regDir+'ToT.reg'
elReg = regDir+'EL.reg'
wlReg = regDir+'WL.reg'

regions = [totReg, elReg, wlReg]

outTables = [ToTDir+'ToTFluxes'+aut+'.tbl',ELobeDir+'ELFluxes'+aut+'.tbl',WLobeDir+'WLFluxes'+aut+'.tbl']

flCorr = np.array(tableFileNames.columns[1],dtype=float)
errPerc=  np.array(tableFileNames.columns[2],dtype=float)


print '\t Executing FileList+Region Combo' 

fr=[]
flIntArray= np.zeros([len(regions),len(tableFileNames.columns[0])])
flErrArray= np.zeros([len(regions),len(tableFileNames.columns[0])])

cutoff= np.zeros([len(tableFileNames.columns[0])])

for i in xrange(0,len(tableFileNames.columns[0])):
    fileName = imDir+tableFileNames.columns[0][i]

    #fileName = fint.cleanHead(fileName)
    print fileName
    datas,heads = fint.openFile(fileName)
    datas=np.squeeze(datas)
    #datas -= noiseValues.columns[9][i]
    #print noiseValues.columns[9][i]
    if not 'CDELT2' in heads:
        heads['CDELT2'] = heads['CD2_2']
        heads['CDELT1'] = heads['CD1_1']
    #if 'Frequency' in tableFileNames.dtype.names:
    #    freq = float(tableFileNames.columns[1][i])*1e6
    #    fr.append(freq)
    #elif 'FREQ' in heads:
    #    freq = float(heads['FREQ'])
    #    fr.append(freq)
    #elif 'CRVAL3' in heads:
    #    freq = float(heads['CRVAL3'])
    #    fr.append(freq)
    #else:
    freq = frList[i]*1e6
    fr.append(freq)

    for j in xrange (0,len(regions)):
        
        if j == 0:
            maskedDataToT, background, noise, pixels=fint.maskDatReg(datas,heads,regions[j],cutoff[i])
            fluxint, numPixBeam =fint.measFlux(maskedDataToT,heads,errPerc[i],'none')
            maskedDataEL = maskedDataToT.copy()
            maskedDataWL = maskedDataToT.copy()
        elif j ==1:
            maskedData, background, noise, pixels = fint.maskDatReg(maskedDataEL,heads,regions[j],cutoff[j])
            fluxint, numPixBeam =fint.measFlux(maskedData,heads,errPerc[i],'none')
        elif j ==2:
            maskedData, background, noise, pixels = fint.maskDatReg(maskedDataWL,heads,regions[j],cutoff[j])
            fluxint, numPixBeam =fint.measFlux(maskedData,heads,errPerc[i],'none')
        
        fluxint *= flCorr[i]
    
        flIntArray[j,i] = fluxint
        t,flErr = fint.writeTable(heads,fluxint,noiseValues.columns[6+j][i],numPixBeam,freq,errPerc[i],outTables[j])
        flErrArray[j,i] = flErr

tName = syn.writeSynageTable(synToTtable,fr,flIntArray[0,:],flErrArray[0,:],'ToT')

tName = syn.writeSynageTable(synETable,fr,flIntArray[1,:],flErrArray[1,:],'ELobe')

tName = syn.writeSynageTable(synWTable,fr,flIntArray[2,:],flErrArray[2,:],'WLobe')

print '''\t+---------+\n\t Fluxes measured\n\t Tables Saved\n\t+---------+'''
  